In [9]:
import os
import sys
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from scipy.io import loadmat

In [10]:
from pyod.models.pca import PCA
from pyod.models.mcd import MCD
from pyod.models.ocsvm import OCSVM
from pyod.models.lof import LOF
from pyod.models.cblof import CBLOF
from pyod.models.knn import KNN
from pyod.models.hbos import HBOS
from pyod.models.abod import ABOD
from pyod.models.iforest import IForest
from pyod.models.feature_bagging import FeatureBagging

In [11]:
from pyod.utils.utility import standardizer
from pyod.utils.utility import precision_n_scores
from sklearn.metrics import roc_auc_score

In [12]:
mat_file_list = ['Anamoly_detec_data-20200815T055304Z-001/Anamoly_detec_data/arrhythmia.mat',
                 'Anamoly_detec_data-20200815T055304Z-001/Anamoly_detec_data/cardio.mat',
                 'Anamoly_detec_data-20200815T055304Z-001/Anamoly_detec_data/glass.mat',
                 'Anamoly_detec_data-20200815T055304Z-001/Anamoly_detec_data/ionosphere.mat',
                 'Anamoly_detec_data-20200815T055304Z-001/Anamoly_detec_data/letter.mat',
                 'Anamoly_detec_data-20200815T055304Z-001/Anamoly_detec_data/lympho.mat',
                 'Anamoly_detec_data-20200815T055304Z-001/Anamoly_detec_data/mnist.mat',
                'Anamoly_detec_data-20200815T055304Z-001/Anamoly_detec_data/musk.mat',
                 'Anamoly_detec_data-20200815T055304Z-001/Anamoly_detec_data/optdigits.mat',
                 'Anamoly_detec_data-20200815T055304Z-001/Anamoly_detec_data/pendigits.mat',
                 'Anamoly_detec_data-20200815T055304Z-001/Anamoly_detec_data/pima.mat',
                 'Anamoly_detec_data-20200815T055304Z-001/Anamoly_detec_data/satellite.mat',
                 'Anamoly_detec_data-20200815T055304Z-001/Anamoly_detec_data/satimage-2.mat',
                 'Anamoly_detec_data-20200815T055304Z-001/Anamoly_detec_data/shuttle.mat',
                'Anamoly_detec_data-20200815T055304Z-001/Anamoly_detec_data/vertebral.mat',
                 'Anamoly_detec_data-20200815T055304Z-001/Anamoly_detec_data/vowels.mat',
                 'Anamoly_detec_data-20200815T055304Z-001/Anamoly_detec_data/wbc.mat']

In [13]:
data = loadmat('Anamoly_detec_data-20200815T055304Z-001/Anamoly_detec_data/cardio.mat')

In [14]:
data

{'__header__': b'MATLAB 5.0 MAT-file, written by Octave 3.8.0, 2014-12-18 10:48:09 UTC',
 '__version__': '1.0',
 '__globals__': [],
 'X': array([[ 0.00491231,  0.69319077, -0.20364049, ...,  0.23149795,
         -0.28978574, -0.49329397],
        [ 0.11072935, -0.07990259, -0.20364049, ...,  0.09356344,
         -0.25638541, -0.49329397],
        [ 0.21654639, -0.27244466, -0.20364049, ...,  0.02459619,
         -0.25638541,  1.14001753],
        ...,
        [-0.41835583, -0.91998844, -0.16463485, ..., -1.49268341,
          0.24461959, -0.49329397],
        [-0.41835583, -0.91998844, -0.15093411, ..., -1.42371616,
          0.14441859, -0.49329397],
        [-0.41835583, -0.91998844, -0.20364049, ..., -1.28578165,
          3.58465295, -0.49329397]]),
 'y': array([[0.],
        [0.],
        [0.],
        ...,
        [1.],
        [1.],
        [1.]])}

In [15]:
len(data)

5

In [20]:
data.keys()

dict_keys(['__header__', '__version__', '__globals__', 'X', 'y'])

In [21]:
data.values()

dict_values([b'MATLAB 5.0 MAT-file, written by Octave 3.8.0, 2014-12-18 10:48:09 UTC', '1.0', [], array([[ 0.00491231,  0.69319077, -0.20364049, ...,  0.23149795,
        -0.28978574, -0.49329397],
       [ 0.11072935, -0.07990259, -0.20364049, ...,  0.09356344,
        -0.25638541, -0.49329397],
       [ 0.21654639, -0.27244466, -0.20364049, ...,  0.02459619,
        -0.25638541,  1.14001753],
       ...,
       [-0.41835583, -0.91998844, -0.16463485, ..., -1.49268341,
         0.24461959, -0.49329397],
       [-0.41835583, -0.91998844, -0.15093411, ..., -1.42371616,
         0.14441859, -0.49329397],
       [-0.41835583, -0.91998844, -0.20364049, ..., -1.28578165,
         3.58465295, -0.49329397]]), array([[0.],
       [0.],
       [0.],
       ...,
       [1.],
       [1.],
       [1.]])])

In [25]:
type(data['X']),data['X'].shape

(numpy.ndarray, (1831, 21))

In [28]:
type(data['y']),data['y'].shape

(numpy.ndarray, (1831, 1))

In [16]:
df_columns = ['Data','#Samples','#Dimensions','Outlier Perc','ABOD','CBOF','FB','HBOS','IForest','KNN','LOF','MCD','OCSVM','PCA']

In [17]:
roc_df=pd.DataFrame(columns=df_columns)
prn_df=pd.DataFrame(columns=df_columns)
time_df=pd.DataFrame(columns=df_columns)
print(roc_df,prn_df,time_df)

Empty DataFrame
Columns: [Data, #Samples, #Dimensions, Outlier Perc, ABOD, CBOF, FB, HBOS, IForest, KNN, LOF, MCD, OCSVM, PCA]
Index: [] Empty DataFrame
Columns: [Data, #Samples, #Dimensions, Outlier Perc, ABOD, CBOF, FB, HBOS, IForest, KNN, LOF, MCD, OCSVM, PCA]
Index: [] Empty DataFrame
Columns: [Data, #Samples, #Dimensions, Outlier Perc, ABOD, CBOF, FB, HBOS, IForest, KNN, LOF, MCD, OCSVM, PCA]
Index: []


In [31]:
from time import time
random_state = np.random.RandomState(42)

for mat_file in mat_file_list:
   print("\n... Processing", mat_file, '...')
   mat = loadmat(os.path.join(mat_file))

   X = mat['X']
   y = mat['y'].ravel()
   outliers_fraction = np.count_nonzero(y) / len(y)
   outliers_percentage = round(outliers_fraction * 100, ndigits=4)

   # construct containers for saving results
   roc_list = [mat_file[:-4], X.shape[0], X.shape[1], outliers_percentage]
   prn_list = [mat_file[:-4], X.shape[0], X.shape[1], outliers_percentage]
   time_list = [mat_file[:-4], X.shape[0], X.shape[1], outliers_percentage]

   # 60% data for training and 40% for testing
   X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4,
                                                       random_state=random_state)

   # standardizing data for processing
   X_train_norm, X_test_norm = standardizer(X_train, X_test)

   classifiers = {'Angle-based Outlier Detector (ABOD)': ABOD(
       contamination=outliers_fraction),
       'Cluster-based Local Outlier Factor': CBLOF(
           contamination=outliers_fraction, check_estimator=True,
           random_state=random_state),
       'Feature Bagging': FeatureBagging(contamination=outliers_fraction,
                                         random_state=random_state),
       'Histogram-base Outlier Detection (HBOS)': HBOS(
           contamination=outliers_fraction),
       'Isolation Forest': IForest(contamination=outliers_fraction,
                                   random_state=random_state),
       'K Nearest Neighbors (KNN)': KNN(contamination=outliers_fraction),
       'Local Outlier Factor (LOF)': LOF(
           contamination=outliers_fraction),
       'Minimum Covariance Determinant (MCD)': MCD(
           contamination=outliers_fraction, random_state=random_state),
       'One-class SVM (OCSVM)': OCSVM(contamination=outliers_fraction),
       'Principal Component Analysis (PCA)': PCA(
           contamination=outliers_fraction, random_state=random_state),
   }

   for clf_name, clf in classifiers.items():
       t0 = time()
       clf.fit(X_train_norm)
       test_scores = clf.decision_function(X_test_norm)
       t1 = time()
       duration = round(t1 - t0, ndigits=4)
       time_list.append(duration)

       roc = round(roc_auc_score(y_test, test_scores), ndigits=4)
       prn = round(precision_n_scores(y_test, test_scores), ndigits=4)

       print('{clf_name} ROC:{roc}, precision @ rank n:{prn}, '
             'execution time: {duration}s'.format(
           clf_name=clf_name, roc=roc, prn=prn, duration=duration))

       roc_list.append(roc)
       prn_list.append(prn)

   temp_df = pd.DataFrame(time_list).transpose()
   temp_df.columns = df_columns
   time_df = pd.concat([time_df, temp_df], axis=0)

   temp_df = pd.DataFrame(roc_list).transpose()
   temp_df.columns = df_columns
   roc_df = pd.concat([roc_df, temp_df], axis=0)

   temp_df = pd.DataFrame(prn_list).transpose()
   temp_df.columns = df_columns
   prn_df = pd.concat([prn_df, temp_df], axis=0)



... Processing Anamoly_detec_data-20200815T055304Z-001/Anamoly_detec_data/arrhythmia.mat ...
Angle-based Outlier Detector (ABOD) ROC:0.7687, precision @ rank n:0.3571, execution time: 0.3199s


C:\Users\LENOVO\anaconda3\lib\site-packages\sklearn\utils\estimator_checks.py:788: ConvergenceWarning: Number of distinct clusters (4) found smaller than n_clusters (8). Possibly due to duplicate points in X.
  estimator1.fit(X, y=y, sample_weight=np.ones(shape=len(y)))
C:\Users\LENOVO\anaconda3\lib\site-packages\sklearn\utils\estimator_checks.py:789: ConvergenceWarning: Number of distinct clusters (4) found smaller than n_clusters (8). Possibly due to duplicate points in X.
  estimator2.fit(X, y=y, sample_weight=None)


ValueError: Buffer dtype mismatch, expected 'INT' but got 'long long'

Exception ignored in: 'sklearn.cluster._k_means_fast._assign_labels_csr'
ValueError: Buffer dtype mismatch, expected 'INT' but got 'long long'


Cluster-based Local Outlier Factor ROC:0.7684, precision @ rank n:0.4643, execution time: 7.7878s
Feature Bagging ROC:0.7799, precision @ rank n:0.5, execution time: 0.9715s
Histogram-base Outlier Detection (HBOS) ROC:0.8511, precision @ rank n:0.5714, execution time: 0.1203s
Isolation Forest ROC:0.8527, precision @ rank n:0.5714, execution time: 0.8871s
K Nearest Neighbors (KNN) ROC:0.782, precision @ rank n:0.5, execution time: 0.144s
Local Outlier Factor (LOF) ROC:0.7787, precision @ rank n:0.4643, execution time: 0.12s


...

Cluster-based Local Outlier Factor ROC:0.9278, precision @ rank n:0.4118, execution time: 6.069s
Feature Bagging ROC:0.9385, precision @ rank n:0.3529, execution time: 0.6049s
Histogram-base Outlier Detection (HBOS) ROC:0.6758, precision @ rank n:0.1765, execution time: 0.014s
Isolation Forest ROC:0.6933, precision @ rank n:0.1765, execution time: 1.2217s
K Nearest Neighbors (KNN) ROC:0.9568, precision @ rank n:0.5294, execution time: 0.1629s
Local Outlier Factor (LOF) ROC:0.9345, precision @ rank n:0.4118, execution time: 0.058s
Minimum Covariance Determinant (MCD) ROC:0.6779, precision @ rank n:0.0, execution time: 1.4983s
One-class SVM (OCSVM) ROC:0.7415, precision @ rank n:0.2941, execution time: 0.077s
Principal Component Analysis (PCA) ROC:0.5787, precision @ rank n:0.1176, execution time: 0.004s

... Processing Anamoly_detec_data-20200815T055304Z-001/Anamoly_detec_data/wbc.mat ...
Angle-based Outlier Detector (ABOD) ROC:0.9232, precision @ rank n:0.3, execution time: 0.168s


C:\Users\LENOVO\anaconda3\lib\site-packages\sklearn\utils\estimator_checks.py:788: ConvergenceWarning: Number of distinct clusters (4) found smaller than n_clusters (8). Possibly due to duplicate points in X.
  estimator1.fit(X, y=y, sample_weight=np.ones(shape=len(y)))
C:\Users\LENOVO\anaconda3\lib\site-packages\sklearn\utils\estimator_checks.py:789: ConvergenceWarning: Number of distinct clusters (4) found smaller than n_clusters (8). Possibly due to duplicate points in X.
  estimator2.fit(X, y=y, sample_weight=None)


ValueError: Buffer dtype mismatch, expected 'INT' but got 'long long'

Exception ignored in: 'sklearn.cluster._k_means_fast._assign_labels_csr'
ValueError: Buffer dtype mismatch, expected 'INT' but got 'long long'


Cluster-based Local Outlier Factor ROC:0.9063, precision @ rank n:0.6, execution time: 5.5304s
Feature Bagging ROC:0.9415, precision @ rank n:0.5, execution time: 0.142s
Histogram-base Outlier Detection (HBOS) ROC:0.9592, precision @ rank n:0.7, execution time: 0.0184s
Isolation Forest ROC:0.9451, precision @ rank n:0.5, execution time: 0.6019s
K Nearest Neighbors (KNN) ROC:0.9437, precision @ rank n:0.5, execution time: 0.0364s
Local Outlier Factor (LOF) ROC:0.9352, precision @ rank n:0.4, execution time: 0.0157s
Minimum Covariance Determinant (MCD) ROC:0.8986, precision @ rank n:0.4, execution time: 0.1464s
One-class SVM (OCSVM) ROC:0.9408, precision @ rank n:0.5, execution time: 0.008s
Principal Component Analysis (PCA) ROC:0.9324, precision @ rank n:0.6, execution time: 0.007s


In [27]:
# print(roc_df,prn_df,time_df)
roc_df

,Data,#Samples,#Dimensions,Outlier Perc,ABOD,CBOF,FB,HBOS,IForest,KNN,LOF,MCD,OCSVM,PCA
0,Anamoly_detec_data-20200815T055304Z-001/Anamol...,452,274,14.6018,0.7687,0.7684,0.7799,0.8511,0.8527,0.782,0.7787,0.8228,0.7986,0.7997
0,Anamoly_detec_data-20200815T055304Z-001/Anamol...,1831,21,9.6122,0.5763,0.8221,0.4879,0.8453,0.9414,0.6959,0.4715,0.8778,0.9507,0.9638
0,Anamoly_detec_data-20200815T055304Z-001/Anamol...,214,9,4.2056,0.7104,0.8506,0.7043,0.6524,0.7195,0.7805,0.7774,0.7165,0.6189,0.622
0,Anamoly_detec_data-20200815T055304Z-001/Anamol...,351,33,35.8974,0.9004,0.8952,0.8933,0.5195,0.8309,0.9134,0.8989,0.9399,0.8372,0.7971
0,Anamoly_detec_data-20200815T055304Z-001/Anamol...,1600,32,6.25,0.8465,0.7423,0.866,0.5728,0.5778,0.845,0.8409,0.7499,0.5744,0.48
0,Anamoly_detec_data-20200815T055304Z-001/Anamol...,148,18,4.0541,0.9382,0.9709,0.9673,0.9964,0.9855,0.9636,0.9636,0.9164,0.9636,0.9818
0,Anamoly_detec_data-20200815T055304Z-001/Anamol...,7603,100,9.2069,0.7813,0.8447,0.7259,0.5675,0.7801,0.8409,0.7085,0.863,0.8417,0.8396
0,Anamoly_detec_data-20200815T055304Z-001/Anamol...,3062,166,3.1679,0.0809,1,0.5228,0.9999,0.9996,0.7348,0.5323,1,1,1
0,Anamoly_detec_data-20200815T055304Z-001/Anamol...,5216,64,2.8758,0.4428,0.7852,0.4641,0.8822,0.5764,0.3824,0.4584,0.3486,0.4972,0.504
0,Anamoly_detec_data-20200815T055304Z-001/Anamol...,6870,16,2.2707,0.7008,0.9609,0.4687,0.9294,0.9422,0.7602,0.481,0.8271,0.93,0.9332


In [29]:
prn_df

,Data,#Samples,#Dimensions,Outlier Perc,ABOD,CBOF,FB,HBOS,IForest,KNN,LOF,MCD,OCSVM,PCA
0,Anamoly_detec_data-20200815T055304Z-001/Anamol...,452,274,14.6018,0.3571,0.4643,0.5,0.5714,0.5714,0.5,0.4643,0.4286,0.5,0.5
0,Anamoly_detec_data-20200815T055304Z-001/Anamol...,1831,21,9.6122,0.1875,0.4844,0.1406,0.4688,0.5,0.2812,0.125,0.3906,0.5938,0.6875
0,Anamoly_detec_data-20200815T055304Z-001/Anamol...,214,9,4.2056,0.25,0.25,0.25,0,0.25,0.25,0.25,0,0.25,0.25
0,Anamoly_detec_data-20200815T055304Z-001/Anamol...,351,33,35.8974,0.8214,0.8036,0.75,0.3393,0.6607,0.8393,0.75,0.8571,0.7143,0.5893
0,Anamoly_detec_data-20200815T055304Z-001/Anamol...,1600,32,6.25,0.275,0.175,0.4,0.125,0.05,0.3,0.325,0.075,0.1,0.05
0,Anamoly_detec_data-20200815T055304Z-001/Anamol...,148,18,4.0541,0.4,0.6,0.6,0.8,0.6,0.6,0.6,0.6,0.6,0.8
0,Anamoly_detec_data-20200815T055304Z-001/Anamol...,7603,100,9.2069,0.3562,0.4007,0.3664,0.1199,0.2979,0.4144,0.339,0.3973,0.3801,0.3767
0,Anamoly_detec_data-20200815T055304Z-001/Anamol...,3062,166,3.1679,0.0333,1,0.1667,0.9667,0.9333,0.2333,0.1333,0.9667,1,1
0,Anamoly_detec_data-20200815T055304Z-001/Anamol...,5216,64,2.8758,0.0161,0,0.0484,0.2581,0.0161,0,0.0484,0,0,0
0,Anamoly_detec_data-20200815T055304Z-001/Anamol...,6870,16,2.2707,0.0308,0.3077,0.0462,0.2615,0.2769,0.0462,0.0462,0.0615,0.2923,0.3385


In [30]:
time_df

,Data,#Samples,#Dimensions,Outlier Perc,ABOD,CBOF,FB,HBOS,IForest,KNN,LOF,MCD,OCSVM,PCA
0,Anamoly_detec_data-20200815T055304Z-001/Anamol...,452,274,14.6018,4.6673,4.3195,0.9196,3.5435,1.0663,0.168,0.128,2.774,0.088,0.242
0,Anamoly_detec_data-20200815T055304Z-001/Anamol...,1831,21,9.6122,0.8843,0.3509,1.4045,0.008,0.696,0.28,0.1432,1.5633,0.1406,0.0877
0,Anamoly_detec_data-20200815T055304Z-001/Anamol...,214,9,4.2056,0.0786,0.0781,0.0719,0.0156,0.4762,0.0156,0,0.2582,0.008,0.0174
0,Anamoly_detec_data-20200815T055304Z-001/Anamol...,351,33,35.8974,0.1686,0.112,0.1317,0.016,0.5708,0.0156,0,0.125,0.0156,0.0689
0,Anamoly_detec_data-20200815T055304Z-001/Anamol...,1600,32,6.25,0.7409,0.2632,1.5134,0.0156,0.7043,0.3306,0.1562,2.1093,0.125,0
0,Anamoly_detec_data-20200815T055304Z-001/Anamol...,148,18,4.0541,0.0822,0.088,0.056,0.008,0.5773,0.0156,0,0.0469,0,0
0,Anamoly_detec_data-20200815T055304Z-001/Anamol...,7603,100,9.2069,13.0867,1.946,78.38,0.0781,3.7348,10.3766,9.5957,5.4846,6.6607,0.2943
0,Anamoly_detec_data-20200815T055304Z-001/Anamol...,3062,166,3.1679,3.7676,0.6406,20.626,0.1094,2.0771,2.9239,2.8656,22.929,1.5924,0.2153
0,Anamoly_detec_data-20200815T055304Z-001/Anamol...,5216,64,2.8758,4.5169,0.9302,22.3984,0.0625,1.5303,3.1123,2.3183,3.3371,2.0834,0.0781
0,Anamoly_detec_data-20200815T055304Z-001/Anamol...,6870,16,2.2707,3.1421,0.5094,7.9615,0.0156,1.3327,1.701,0.9279,3.7479,2.0299,0.0156
